# Save and Load Models - Tensorflow

- <a href='https://www.tensorflow.org/tutorials/keras/save_and_load'>*Tensorflow Documentation / Tutorial*</a>

## Setup

In [1]:
import os
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras import layers, models

tf.random.set_seed(51)

In [2]:
# Load in MNIST dataset.
((train_images, train_labels), 
 (test_images, test_labels)) = tf.keras.datasets.mnist.load_data()

train_images = train_images[:1500]
train_labels = train_labels[:1500]
test_images = test_images[:500]
test_labels = test_labels[:500]

# Calculate resizing dimension (28 x 28).
DIM = 28**2

# Flatten images and rescale pixels.
train_images = train_images.reshape(-1, DIM) / 255.0
test_images = test_images.reshape(-1, DIM) / 255.0

In [3]:
train_images.shape, test_images.shape, train_labels.shape, test_labels.shape

((1500, 784), (500, 784), (1500,), (500,))

In [4]:
# One Hot Encode target.
ohe = OneHotEncoder(sparse=False)
train_labels_ohe = ohe.fit_transform(train_labels.reshape(-1, 1))
test_labels_ohe = ohe.transform(test_labels.reshape(-1, 1))

In [5]:
train_labels_ohe.shape, test_labels_ohe.shape

((1500, 10), (500, 10))

In [6]:
# Define a simple sequential model
def create_model():
    """Create and return a compiled model."""
    
    model = models.Sequential()
    model.add(layers.Dense(512, activation='relu', input_dim=DIM))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
        )

    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 435,402
Trainable params: 435,402
Non-trainable params: 0
_________________________________________________________________


# `callbacks.ModelCheckpoint`
- https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [8]:
checkpoint_path = 'test_model_weights/cp.ckpt'

# Create a ModelCheckpoint
## filepath
## save_weights_only
## save_best_only
checkpoint = ModelCheckpoint(
    checkpoint_path,
    save_weights_only=True,
    save_best_only=True,
    verbose=1
)

checkpoint

In [9]:
# Fit the model
## train
## test
## batch
## epochs
## validation_data
## callbacks
model.fit(
    train_images,
    train_labels_ohe,
    batch_size=64,
    epochs=10,
    validation_data=(test_images, test_labels_ohe),
    callbacks=[checkpoint]
)

Epoch 1/10
24/24 [==============================] - 1s 32ms/step - loss: 1.7002 - accuracy: 0.4696 - val_loss: 0.6577 - val_accuracy: 0.8100

Epoch 00001: val_loss improved from inf to 0.65773, saving model to test_model_weights/cp.ckpt
Epoch 2/10
24/24 [==============================] - 0s 6ms/step - loss: 0.4686 - accuracy: 0.8638 - val_loss: 0.4038 - val_accuracy: 0.8840

Epoch 00002: val_loss improved from 0.65773 to 0.40383, saving model to test_model_weights/cp.ckpt
Epoch 3/10
24/24 [==============================] - 0s 6ms/step - loss: 0.2452 - accuracy: 0.9348 - val_loss: 0.4019 - val_accuracy: 0.8840

Epoch 00003: val_loss improved from 0.40383 to 0.40189, saving model to test_model_weights/cp.ckpt
Epoch 4/10
24/24 [==============================] - 0s 6ms/step - loss: 0.1907 - accuracy: 0.9504 - val_loss: 0.3491 - val_accuracy: 0.8980

Epoch 00004: val_loss improved from 0.40189 to 0.34912, saving model to test_model_weights/cp.ckpt
Epoch 5/10
24/24 [=========================

In [10]:
if os.listdir('test_model_weights'):
    print("There's something in here")
else:
    print("It's empty")

There's something in here


In [11]:
# Look inside save folder.
os.listdir('test_model_weights')

['cp.ckpt.data-00000-of-00001',
 'checkpoint',
 '.ipynb_checkpoints',
 'cp.ckpt.index']

In [12]:
checkpoint_path_formatted = 'test_model_weights/cp-{epoch:04d}.ckpt' # (uses `str.format`)

# Fit the model again - add formatted string as path.
checkpoint = ModelCheckpoint(
    checkpoint_path_formatted,
    save_weights_only=True,
    save_best_only=True,
    verbose=1
)

model = create_model()
model.fit(
    train_images,
    train_labels_ohe,
    batch_size=64,
    epochs=10,
    validation_data=(test_images, test_labels_ohe),
    callbacks=[checkpoint]
)

Epoch 1/10
24/24 [==============================] - 1s 11ms/step - loss: 1.6907 - accuracy: 0.4764 - val_loss: 0.6467 - val_accuracy: 0.7900

Epoch 00001: val_loss improved from inf to 0.64671, saving model to test_model_weights/cp-0001.ckpt
Epoch 2/10
24/24 [==============================] - 0s 5ms/step - loss: 0.4572 - accuracy: 0.8714 - val_loss: 0.4314 - val_accuracy: 0.8660

Epoch 00002: val_loss improved from 0.64671 to 0.43135, saving model to test_model_weights/cp-0002.ckpt
Epoch 3/10
24/24 [==============================] - 0s 5ms/step - loss: 0.2404 - accuracy: 0.9385 - val_loss: 0.3910 - val_accuracy: 0.8800

Epoch 00003: val_loss improved from 0.43135 to 0.39105, saving model to test_model_weights/cp-0003.ckpt
Epoch 4/10
24/24 [==============================] - 0s 5ms/step - loss: 0.1767 - accuracy: 0.9595 - val_loss: 0.3776 - val_accuracy: 0.8880

Epoch 00004: val_loss improved from 0.39105 to 0.37763, saving model to test_model_weights/cp-0004.ckpt
Epoch 5/10
24/24 [=====

In [13]:
# Look inside save folder.
sorted(os.listdir('test_model_weights'))

['.ipynb_checkpoints',
 'checkpoint',
 'cp-0001.ckpt.data-00000-of-00001',
 'cp-0001.ckpt.index',
 'cp-0002.ckpt.data-00000-of-00001',
 'cp-0002.ckpt.index',
 'cp-0003.ckpt.data-00000-of-00001',
 'cp-0003.ckpt.index',
 'cp-0004.ckpt.data-00000-of-00001',
 'cp-0004.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0007.ckpt.data-00000-of-00001',
 'cp-0007.ckpt.index',
 'cp.ckpt.data-00000-of-00001',
 'cp.ckpt.index']

In [14]:
def evaluate_nn(model, X_test, y_test):
    """Print model accuracy on test set."""
    
    loss, acc = model.evaluate(X_test, y_test)
    print(f'Model Accuracy:\n\t{round(acc, 3)}')

    
# Evaluate model
evaluate_nn(model, test_images, test_labels_ohe)

16/16 [==============================] - 0s 1ms/step - loss: 0.3105 - accuracy: 0.9080
Model Accuracy:
	0.908


In [15]:
# Create new model and evaluate before training.
new_model = create_model()

evaluate_nn(new_model, test_images, test_labels_ohe)

16/16 [==============================] - 0s 1ms/step - loss: 2.3835 - accuracy: 0.0865
Model Accuracy:
	0.1


In [16]:
# Load weights from `checkpoint_path`
new_model.load_weights('test_model_weights/cp-0007.ckpt')

# Evaluate model.
evaluate_nn(new_model, test_images, test_labels_ohe)

16/16 [==============================] - 0s 1ms/step - loss: 0.3102 - accuracy: 0.9040
Model Accuracy:
	0.904


In [17]:
# Trying to train on top of the pretrained network.
new_model.fit(
    train_images,
    train_labels_ohe,
    batch_size=64,
    epochs=5,
    validation_data=(test_images, test_labels_ohe),
    callbacks=[checkpoint]
)

Epoch 1/5
24/24 [==============================] - 0s 7ms/step - loss: 0.0386 - accuracy: 0.9967 - val_loss: 0.3181 - val_accuracy: 0.9000

Epoch 00001: val_loss did not improve from 0.31023
Epoch 2/5
24/24 [==============================] - 0s 5ms/step - loss: 0.0280 - accuracy: 0.9993 - val_loss: 0.3277 - val_accuracy: 0.9080

Epoch 00002: val_loss did not improve from 0.31023
Epoch 3/5
24/24 [==============================] - 0s 5ms/step - loss: 0.0203 - accuracy: 0.9993 - val_loss: 0.3284 - val_accuracy: 0.9120

Epoch 00003: val_loss did not improve from 0.31023
Epoch 4/5
24/24 [==============================] - 0s 5ms/step - loss: 0.0150 - accuracy: 0.9993 - val_loss: 0.3218 - val_accuracy: 0.9140

Epoch 00004: val_loss did not improve from 0.31023
Epoch 5/5
24/24 [==============================] - 0s 5ms/step - loss: 0.0116 - accuracy: 1.0000 - val_loss: 0.3189 - val_accuracy: 0.9140

Epoch 00005: val_loss did not improve from 0.31023


# `''model''.save()` - `models.load_model`

In [18]:
saved_model_path = 'test_model_weights/saved_model.h5'

# Save original model.
new_model.save(saved_model_path)

# Delete model from memory (`del model`)
del new_model

In [19]:
# Load model from path and evaluate.
new_model = models.load_model(saved_model_path)

In [20]:
evaluate_nn(new_model, test_images, test_labels_ohe)

16/16 [==============================] - 0s 931us/step - loss: 0.3189 - accuracy: 0.9140
Model Accuracy:
	0.914


# Setting Up Conditionals for Training

- *Many different options for conditionals. Pick something that suits your needs.*

```python
# # Modeling
# # # 


# Example function.
def check_if_file_in_dir(f_name, directory):
    if f_name in os.listdir():
        return True
    return False


#####


# Example if / else.
# For checking if the model exists.
if 'model_1.h5' in os.listdir():
    model = models.load_model('model_1.h5')
else:
    model = create_model()
    model.fit(...)
    model.save('model_1.h5')

    
#####

    
# Example global.
# Create global variable in the notebook.
TRAIN_MY_MODEL = False

if TRAIN_MY_MODEL:
    model = create_model()
    model.fit(...)
    model.save('saved_model.h5')
else:
    model = models.load_model('saved_model.h5')
```